In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import pandas as pd
import os
from os.path import join

from notebooks.jupyter.utils import (
    save_parquet_excel,
    check_crs,
    get_data_diretorio,
    diretorio,
)
from notebooks.jupyter.utils.downloads import(
    download_malha_geosampa,
)

# Arborização Viária

## Importar dados 

In [3]:
data_path = get_data_diretorio()

In [4]:
cache_path = join(
    data_path, 
    'cache'
)
cache_full_path = os.path.join(
        cache_path,
        'arvore_concat.parquet'
    )
if os.path.exists(
    cache_full_path
):
    print('Carregando arquivo em cache')
    gdf_arvore = gpd.read_parquet(cache_full_path)
else:
    gdf_arvore = download_malha_geosampa('arvore', cache_path, True)

Carregando arquivo em cache


In [5]:
gdf_arvore.shape

(652976, 22)

In [6]:
gdf_arvore.sample(3)

,cd_identif,nm_vulgar_,dc_nm_popu,cd_especie,dc_especie,cd_identi0,cd_tipo_po,dc_tipo_po,qt_diametr,qt_altura_,...,dc_tipo_es,nm_logrado,nr_logrado,sg_distrit,nm_distrit,nm_subpref,cd_inspeca,dt_inspeca,dt_atualiz,geometry
143436,228417,None,None,NaN,None,None,NaN,None,NaN,NaN,...,None,None,NaN,SAU,SAUDE,VILA MARIANA,NaN,NaT,NaT,POINT (333779.109 7388193.652)
407413,466009,None,None,NaN,None,None,NaN,None,NaN,NaN,...,None,None,NaN,VSO,VILA SONIA,BUTANTA,NaN,NaT,NaT,POINT (321549.162 7387416.132)
329705,396074,None,None,NaN,None,None,NaN,None,NaN,NaN,...,None,None,NaN,PQC,PARQUE DO CARMO,ITAQUERA,NaN,NaT,NaT,POINT (352043.171 7393524.837)


## Padronizaçã de colunas

Originalmente, muitas das colunas dos gdfs que serão interseccionados têm nomes repetidos, ou estavam vazias. Para que isso não faça com que a análise das tabelas seja confusa, apagaremos as colunas que julgadas desnecessárias, e renomearemos as necessárias.

In [7]:
drop_arvore_cols = {
    'nm_vulgar_', 
    'dc_nm_popu', 
    'cd_especie', 
    'dc_especie',
    'cd_identi0', 
    'cd_tipo_po', 
    'dc_tipo_po', 
    'qt_diametr', 
    'qt_altura_',
    'qt_diamet0', 
    'cd_tipo_es', 
    'dc_tipo_es', 
    'nm_logrado', 
    'nr_logrado',
    'cd_inspeca',
    'dt_inspeca',
    'dt_atualiz'
}

gdf_arvore.drop(columns=drop_arvore_cols, axis=1, inplace=True)

In [8]:
rename_arvore_cols = {
    'sg_distrit' : 'sg_dist_arv', 
    'nm_distrit' : 'nm_dist_arv'
}

gdf_arvore.rename(rename_arvore_cols, axis=1, inplace=True)

## Conferir CRS

In [9]:
gdf_arvore = check_crs(gdf_arvore)

## Salvar GDF

Será que eu não poderia colocar um if pra, caso tenha sido lido o cache E a já tenha o gdf salvo, só ler ele... acho que faz mais sentido, mas fica depois pra refatoração.

In [10]:
save_parquet_excel(
    gdf_arvore, 
    'malha_arborizacao_viaria',
    data_path,
    data_subpath='assets' 
)

In [11]:
gdf_arvore.sample(3)

,cd_identif,sg_dist_arv,nm_dist_arv,nm_subpref,geometry
374215,436131,SRA,SAO RAFAEL,SAO MATEUS,POINT (349909.519 7385758.637)
497648,547217,JDS,JARDIM SAO LUIS,M BOI MIRIM,POINT (321967.953 7382552.424)
395857,455609,JDH,JARDIM HELENA,SAO MIGUEL,POINT (356022.785 7401629.601)
